# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import networkx as nx
import pygwalker as pyg
import re
import requests
import plotly.express as px
import csv
import pycountry
import base64
import geoip2.database

# Initialise CSV File

In [ ]:
df = pd.read_csv('https://github.com/NorthwaveSecurity/complete_translation_leaked_chats_conti_ransomware/blob/main/jabber_chat_2021_2022_translated.csv?raw=true', index_col=0)
df.rename(columns={'ts':'Timestamp', 'from':'Sender', 'to':'Recipient', 'body':'Message_Body', 'body_en':'Message_English','body_language':'Body_Language'}, inplace=True)
df.head()

# Table of Activity

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp']).dt.date
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

data = df.groupby(df['Timestamp']).count()
data.head()

# Daily Activity

In [ ]:
chat_activity = df.groupby(df['Timestamp'])['Sender'].count()
chat_activity.plot(kind='line',figsize=(10,10),xlabel='Date',ylabel='Number of Messages',legend=True,title="Chat Activity per Day")

# Activity Per User

In [ ]:
count_users = df.groupby(df['Sender'] == "defender@q3mcco35auwcstmt.onion").value_counts()
print(count_users[:20])

# User Activity Comparison

In [ ]:
# List of Users
users=df['Sender'].drop_duplicates()
print(users)

# Messages per User Graph
user_messages = (df['Sender'].value_counts()).iloc[:20]
user_messages.plot(kind='barh',figsize=(10,10),xlabel='User',ylabel='Number of Messages',legend=True,title="Sent Messages Per User")

# Messages Between Users

In [ ]:
G2 = nx.DiGraph()
G2 = nx.from_pandas_edgelist(df, source='Sender', target='Recipient')

G2.nodes()

pos = nx.spring_layout(G2)

nx.draw_networkx_nodes(G2, pos, node_size=20, alpha=1)
nx.draw_networkx_edges(G2, pos)
nx.draw_networkx_labels(G2, pos)
plt.figure(2, figsize=(50,50))
nx.draw(G2,pos, with_labels=True)
#plt.show()

# Word Map

In [ ]:
# Remove encrypted messages
df_no_encrypt = df.drop(df[df['Message_English'] == '[Error: The message is encrypted and cannot be decrypted.]'].index)
df_no_encrypt = df_no_encrypt.drop(df_no_encrypt[df_no_encrypt['Message_English'] == '[ERROR: This message is encrypted, and you are unable to decrypt it.]'].index)

# Combine all messages into one string
messages = " ".join(message for message in df_no_encrypt.Message_English)

# Create list of stop words
stopwords = set(STOPWORDS)
stopwords.update(["Hey", "Error", "bro", "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"])

wordcloud = WordCloud(width=1800, height=1800, stopwords=stopwords).generate(messages)

plt.figure(figsize= (8,8), facecolor=None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


# Making Things Easier with PyGWalker

In [ ]:
walker = pyg.walk(df)

# Extract Key Words

In [ ]:
keywords = ['Fraser', 'Mango', 'bitcoin', 'email']

for keyword in keywords:
    detect_keyword = df_no_encrypt[df_no_encrypt['Message_English'].str.contains(keyword)]
    print("KEYWORD: " + keyword)
    print("RESULTS DETECTED: ")
    print (detect_keyword['Message_English'])

# Extract Bitcoin Addresses

In [ ]:
# Extract Bitcoin addresses
bitcoin_match = re.findall(r'(?:[13]{1}[a-km-zA-HJ-NP-Z1-9]{26,33}|bc1[a-z0-9]{39,59})', messages)
dedup_addresses = list(set(bitcoin_match)) # Remove duplicates

# Write addresses to file
with open('bitcoin_addresses.csv', 'w', newline='') as out_file:
    csv_writer = csv.writer(out_file, quoting=csv.QUOTE_ALL)
    csv_writer.writerows([[address] for address in dedup_addresses])    

# Show addresses in context
columns = df[["Timestamp", "Sender", "Recipient", "Message_English"]]
filtered_df = columns.loc[df['Message_English'].str.contains('(?:[13]{1}[a-km-zA-HJ-NP-Z1-9]{26,33}|bc1[a-z0-9]{39,59})')]
filtered_df.tail()

# Extract Emails

In [ ]:
# Extract Emails
email_match = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', messages)
dedup_emails = list(set(email_match)) # Remove duplicates

# Write addresses to file
with open('email_addresses.csv', 'w', newline='') as out_file:
    csv_writer = csv.writer(out_file, quoting=csv.QUOTE_ALL)
    csv_writer.writerows([[email] for email in dedup_emails])    

# Show addresses in context
columns = df[["Timestamp", "Sender", "Recipient", "Message_English"]]
filtered_df = columns.loc[df['Message_English'].str.contains('[\w.+-]+@[\w-]+\.[\w.-]+')]
filtered_df.tail()

# Extract URLs

In [ ]:
# Extract URLs
url_match = re.findall(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", messages)
dedup_urls = list(set(url_match)) # Remove duplicates

# Write addresses to file
with open('urls.csv', 'w', newline='') as out_file:
    csv_writer = csv.writer(out_file, quoting=csv.QUOTE_ALL)
    csv_writer.writerows([[url] for url in dedup_urls])    

# Show addresses in context
columns = df[["Timestamp", "Sender", "Recipient", "Message_English"]]
filtered_df = columns.loc[df['Message_English'].str.contains('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')]
filtered_df.tail()

# Enrich URLs

In [ ]:
# Enrich URLs with VirusTotal
url_id = base64.urlsafe_b64encode("https://1ty.me/Pmreqmdp1".encode()).decode().strip("=") # Encode URL

url = "https://www.virustotal.com/api/v3/urls/" + url_id

headers = {
    "accept": "application/json",
    "x-apikey": "<INSERT-API-KEY-HERE>"
}

response = requests.get(url, headers=headers)

print(response.text)

# Extract IP Addresses

In [ ]:
# Extract IPs
ip_match = re.findall(r"(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})", messages)
dedup_ips = list(set(ip_match)) # Remove duplicates

# Write addresses to file
with open('ip_addresses.csv', 'w', newline='') as out_file:
    csv_writer = csv.writer(out_file, quoting=csv.QUOTE_ALL)
    csv_writer.writerows([[ip] for ip in dedup_ips])    

# Show addresses in context
columns = df[["Timestamp", "Sender", "Recipient", "Message_English"]]
filtered_df = columns.loc[df['Message_English'].str.contains('(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')]
filtered_df.tail()

# Enrich IP Addresses

In [ ]:
# Enrich IPs with VirusTotal
vt_url = ("https://www.virustotal.com/api/v3/ip_addresses/" + "185.99.133.115")
headers = {"accept": "application/json", "x-apikey": "<INSERT-API-KEY-HERE>"}
response = requests.get(vt_url, headers=headers)
print(response.text)

# Geolocate IP Addresses

In [ ]:
# Geolocate IP addresses
country_list=[]

with geoip2.database.Reader("./GeoLite2-Country.mmdb") as reader:
    for ip in dedup_ips:
        try:
            response = reader.country(ip)
            #print(response)
            #country_list.append(response.country.name)
            #print(pycountry.countries.get(name=response.country.name).alpha_3)
            country_list.append(pycountry.countries.get(name=response.country.name).alpha_3)
        except:
            pass

country_df = pd.DataFrame({'Country':country_list})
country_df['Count'] = country_df.groupby('Country').transform('size')
country_df = country_df.drop_duplicates().sort_values(by=['Count'], ascending=False)

fig = px.choropleth(country_df, locations='Country', color='Count', 
                    color_continuous_scale="Plasma", 
                    range_color=(0, 20))
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Initialising PandasAI

In [ ]:
# Import
from pandasai import SmartDataframe
from pandasai.llm import OpenAI

# Instantiate a LLM
llm = OpenAI(api_token="<INSERT-API-KEY-HERE>")


# Preview the Data

In [ ]:
df = SmartDataframe(df, config={"llm": llm})
df.chat('Show me the first 5 rows of the table')

# Learn About the Data

In [ ]:
df = SmartDataframe(df, config={"llm": llm})
df.chat('How many users and messages are in this chat? On which day are most messges sent?')

# User Activity - PandasAI

In [ ]:
df = SmartDataframe(df, config={"llm": llm})
df.chat('Show me a graph of chat activity over time')

# Top Users - PandasAI

In [ ]:
df = SmartDataframe(df, config={"llm": llm})
df.chat('Show me the top 20 users based on messages in a bar chart')

# User Interactions - PandasAI

In [ ]:
df = SmartDataframe(df, config={"llm": llm, "custom_whitelisted_dependencies":["networkx"]})
df.chat('Show me a node graph of the top 50 users and who they interacted with')

# Word Map - PandasAI

In [ ]:
df_no_encrypt = SmartDataframe(df_no_encrypt, config={"llm": llm, "custom_whitelisted_dependencies":["re", "wordcloud"]})
df_no_encrypt.chat('Create a word cloud of the top words used in the english translated messages')

# Extract Key Words - PandasAI

In [ ]:
df_no_encrypt = SmartDataframe(df_no_encrypt, config={"llm": llm, "custom_whitelisted_dependencies":["re"]})
df_no_encrypt.chat('Find how many times the word bitcoin appears in the english messages')

# Extract Bitcoin Wallet Addresses - PandasAI

In [ ]:
df = SmartDataframe(df, config={"llm": llm, "custom_whitelisted_dependencies":["re"]})
df.chat('Find all Bitcoin Wallet Addresses in the English messages and display them in table format')

# Extract Emails - PandasAI

In [ ]:
df = SmartDataframe(df, config={"llm": llm, "custom_whitelisted_dependencies":["re"]})
df.chat('Find all Email Addresses in the English messages and display them in table format')

# Extract URLs - PandasAI

In [ ]:
df = SmartDataframe(df, config={"llm": llm, "custom_whitelisted_dependencies":["re"]})
df.chat('Find all URLs in the English messages and display them in table format')

# Extract & Enrich IPs - PandasAI

In [ ]:
df = SmartDataframe(df, config={"llm": llm, "custom_whitelisted_dependencies":["re"]})
ip_extract = df.chat('Find all IP Addresses in the English messages and display them in a list')

# Export to CSV
with open('ip_addresses.csv', 'w', newline='') as out_file:
    csv_writer = csv.writer(out_file, quoting=csv.QUOTE_ALL)
    csv_writer.writerows([[ip] for ip in ip_extract])

# Enrich First 5 IPs with VirusTotal
for ip in ip_extract[:5]:
    vt_url = ("https://www.virustotal.com/api/v3/ip_addresses/" + ip)
    headers = {"accept": "application/json", "x-apikey": "<INSERT-API-KEY-HERE>"}
    response = requests.get(vt_url, headers=headers)
    print(response.text)
